In [ ]:
import os
import pims_nd2
import numpy as np
import pytz
import datetime as dt
import time
from matplotlib import pyplot as plt
import labrotation.file_handling as fh

In [ ]:
lv_fpath = fh.open_file("Open labview .txt file")
lv_tstamp_fpath = fh.open_file("Open labview time.txt file")
nik_fpath = fh.open_file("Open nd2 file")

In [ ]:
lv_fname = os.path.split(lv_fpath)[1]
lv_tstamp_fname = os.path.split(lv_tstamp_fpath)[1]
nik_fname = os.path.split(nik_fpath)[1]

In [ ]:
bad_time_txt = True  # important: if only Nikon stamps in time.txt file, then this should be True. 
# In this case, the last modification of the time.txt file cannot be used for correcting the delay between closing .txt and
# the last row entry.

In [ ]:
#folder = "E:\\Nico\\T538\\120minafterSD"
#lv_fname = "M278.240123.1129.txt"
#lv_tstamp_fname = "M278.240123.1129time.txt"
#nik_fname = "T538_120minafterSD_240123_baseline_002.nd2"

#folder = "E:\\TwoPhoton\\tmev\\T301\\T301_tmev_d1"
#lv_fname = "T301_tmev_d1.270820.1110.txt"
#lv_tstamp_fname = "T301_tmev_d1.270820.1110time.txt"
#nik_fname = "T301_tmev_d1.270820.1110.nd2"

#lv_fpath = os.path.join(folder, lv_fname)
#lv_tstamp_fpath = os.path.join(folder, lv_tstamp_fname)
#nik_fpath = os.path.join(folder, nik_fname)

# The Master Plan
Get exact time of first frame of the Nikon recording. This has a corresponding entry in the time.txt second column. Get this time in the labview .txt file by approximating last modification date with time stamp of last entry, and trace back to the time of the first Nikon frame.

In [ ]:
tzone_local = pytz.timezone('Europe/Berlin')
tzone_utc = pytz.utc

## Read out files
### Nikon

In [ ]:
nik = pims_nd2.ND2_Reader(nik_fpath)

In [ ]:
# first element is not zero: after starting recording, the first frame was read slightly afterwards
nik_stamps_ms = np.array([nik[i_frame].metadata["t_ms"] for i_frame in range(len(nik))])

### Labview time.txt

In [ ]:
lv_time_stamps = []  # should be recorded in labview file as ms
reso = []
galvo = []
lfp = []

with open(lv_tstamp_fpath, "r") as f:
    lines = [list(map(lambda x: float(x), row.rstrip().split("\t"))) for row in f.readlines()]
    last_line = lines[-1]
    next_last_line = lines[-2]
    
    lv_time_stamps = np.array([line[0] for line in lines])
    reso = np.array([line[1] for line in lines]) 
    galvo = np.array([line[2] for line in lines])
    lfp = np.array([line[3] for line in lines])

In [ ]:
t_stamps_reso = reso[reso.nonzero()[0]]  # assume resonant scanning
i_t_stamps_reso = reso.nonzero()[0]

### Labview .txt

In [ ]:
lv_tstamps_ms = []  # should be recorded in labview file as ms
lv_speed = []
with open(lv_fpath, "r") as f:
    lines = [list(map(lambda x: int(x), row.rstrip().split("\t"))) for row in f.readlines()]
    last_line = lines[-1]
    next_last_line = lines[-2]
    # decide whether to drop last line in file (last_whole_row is in 1-indexing, perfect for [:last_whole_row] indexing)
    if len(last_line) < len(next_last_line):
        last_whole_row = len(lines) - 1  # skip last line as it is not a properly recorded line
    else:
        last_whole_row = len(lines)  # use whole file
    
    lines = lines[:last_whole_row]
    
    lv_tstamps_ms = np.array([line[8] for line in lines])
    lv_speed = np.array([line[1] for line in lines]) 

## Get times
### Nikon

In [ ]:
t_abs_nik_start = tzone_utc.localize(nik.metadata["time_start_utc"])

In [ ]:
t_abs_nik_first_frame = t_abs_nik_start + dt.timedelta(milliseconds=nik_stamps_ms[0])

In [ ]:
t_abs_nik_last_frame = t_abs_nik_start + dt.timedelta(milliseconds=nik_stamps_ms[-1])

In [ ]:
# we can use time.txt end time to correct for last entry <-> last file modification time delta

### Labview .txt

In [ ]:
t_abs_lv_end = tzone_local.localize(dt.datetime.fromtimestamp(os.path.getmtime(lv_fpath)))
print(t_abs_lv_end)

In [ ]:
t_abs_lv_tstamp_end = tzone_local.localize(dt.datetime.fromtimestamp(os.path.getmtime(lv_tstamp_fpath)))  # time.localtime()
print(t_abs_lv_tstamp_end)

In [ ]:
abs(t_abs_lv_end-t_abs_lv_tstamp_end).total_seconds()  
# should be a tiny difference IF time.txt was properly recorded

In [ ]:
abs(t_abs_nik_last_frame - t_abs_lv_tstamp_end).total_seconds()

### Labview time.txt

In [ ]:
dt_lv_tstamp_end_nik_first_s = (t_abs_lv_tstamp_end - t_abs_nik_first_frame).total_seconds()
print(dt_lv_tstamp_end_nik_first_s)

In [ ]:
t_abs_lv_tstamp_end

In [ ]:
t_abs_nik_last_frame

In [ ]:
dt_lv_tstamp_end_nik_last_s = (t_abs_lv_tstamp_end - t_abs_nik_last_frame).total_seconds()
print(dt_lv_tstamp_end_nik_last_s)

In [ ]:
assert dt_lv_tstamp_end_nik_first_s > 0  # labview end should come after first nikon frame

In [ ]:
# calculate back from last labview .txt entry
if not bad_time_txt:  # first column has normal labview time stamps
    t_lv_tstamps_first_nik_ms = lv_time_stamps[-1] - 1000.*dt_lv_tstamp_end_nik_first_s
else:  # only nikon frames in time.txt, i.e. second column non-zero, first column full zero. Last entry into file is 
    t_lv_tstamps_first_nik_ms = t_stamps_reso[-1] - 1000.*dt_lv_tstamp_end_nik_first_s
print(t_lv_tstamps_first_nik_ms)

### Calculate error in estimation in time.txt
The first Nikon frame is approximated, but also exists in the file. We can compare the two values and apply it to the labview txt file.

In [ ]:
t_lv_tstamps_first_nik_true_ms = t_stamps_reso[0]

In [ ]:
dt_file_end_last_entry_ms =  t_lv_tstamps_first_nik_ms - t_lv_tstamps_first_nik_true_ms
assert dt_file_end_last_entry_ms > 0  # assumption is that labview file ALWAYS gets last modification slightly AFTER writing last entry...

### Get Nikon first frame time in Labview .txt

In [ ]:
dt_lv_end_nik_first_s = (t_abs_lv_end - t_abs_nik_first_frame).total_seconds() 
print(f"{dt_lv_end_nik_first_s/60.}  minutes between end of labview file and first Nikon frame")

In [ ]:
# correct for the fact that the file closing time stamp is always later than the actual last entry time stamp
dt_lv_end_nik_first_s_corrected = dt_lv_end_nik_first_s - dt_file_end_last_entry_ms/1000.

In [ ]:
assert dt_lv_end_nik_first_s_corrected > 0  # labview end should come after first nikon frame

In [ ]:
print(f"{dt_lv_end_nik_first_s_corrected/60.} corrected minutes between end of labview file and first Nikon frame")

In [ ]:
# calculate back from last labview .txt entry
t_lv_first_nik =  lv_tstamps_ms[-1]   - 1000.*dt_lv_end_nik_first_s_corrected
print(f"{t_lv_first_nik} time of first nikon frame in labview")

In [ ]:
# 0-indexing, index of first time stamp that is larger than the nikon starting time
i_nik_start = np.searchsorted(lv_tstamps_ms, t_lv_first_nik)  
print(i_nik_start)

In [ ]:
t_abs_lv_start = t_abs_lv_end - dt.timedelta(seconds=(lv_tstamps_ms[-1]/1000.))

### Get offset between .txt and time.txt

In [ ]:
dt_lv_lvtime = t_lv_tstamps_first_nik_true_ms - t_lv_first_nik
print(dt_lv_lvtime)

In [ ]:
lv_tstamps_ms_shifted = lv_tstamps_ms + dt_lv_lvtime

In [ ]:
lv_tstamps_ms_shifted[i_nik_start]

In [ ]:
#t_stamps_reso
#i_t_stamps_reso

In [ ]:
t_stamps_reso[0]

### Calculate time between first labview .txt/time.txt entry and first nikon frames
They should be roughly equal shouldn't they?

In [ ]:
t_abs_nik_first_frame

In [ ]:
dt_lv_time_s = (reso[-1] - lv_time_stamps[0])/1000.

In [ ]:
t_abs_lv_tstamp_start = t_abs_lv_tstamp_end -  dt.timedelta(seconds=dt_lv_time_s)

In [ ]:
dt_lvtime_start_nik_start = (reso[1] - lv_time_stamps[0])/1000.

In [ ]:
dt_lv_start_nik_start = (t_abs_nik_first_frame - t_abs_lv_start)

In [ ]:
dt_lv_start_nik_start.total_seconds()

In [ ]:
dt_lvtime_start_nik_start

# Create corrected time.txt file

In [ ]:
lv_time_stamps

In [ ]:
len(reso.nonzero()[0])

In [ ]:
# columns are:
# lv_time_stamps
# reso
# galvo
# lfp

In [ ]:
len(lv_tstamps_ms_shifted) 

In [ ]:
len(reso.nonzero()[0])

In [ ]:
len_corr = len(lv_tstamps_ms_shifted) + len(reso.nonzero()[0]) + 1  # first row will be kept in new time.txt
lv_time_stamps_corr = np.zeros(len_corr)
reso_corr = np.zeros(len_corr)
galvo_corr = np.zeros(len_corr)
lfp_corr = np.zeros(len_corr)

In [ ]:
galvo.sum()

In [ ]:
lfp

### Fill first two columns
third and fourth, galvo and lfp, should be empty

In [ ]:
assert lv_tstamps_ms_shifted[-1] > t_stamps_reso[-1]

In [ ]:
# copy first row
lv_time_stamps_corr[0] = lv_time_stamps[0]
reso_corr[0] = reso[0]
galvo_corr[0] = galvo[0]
lfp_corr[0] = lfp[0]

In [ ]:
# loop through both columns simultaneously, insert nikon time stamps (reso) in second column at appropriate locations.
i_tstamp = 0
i_reso = 0
i_new = 1
while i_new < len_corr:
    if lv_tstamps_ms_shifted[i_tstamp] < t_stamps_reso[i_reso]:
        lv_time_stamps_corr[i_new] = lv_tstamps_ms_shifted[i_tstamp]
        i_tstamp += 1
    else:
        reso_corr[i_new] = t_stamps_reso[i_reso]
        i_reso += 1
    if i_reso == len(t_stamps_reso):
        i_new += 1
        break
    i_new += 1

assert i_reso == len(t_stamps_reso)
if i_new < len_corr:
    lv_time_stamps_corr[i_new:] = lv_tstamps_ms_shifted[i_tstamp:]

### Test results

In [ ]:
assert len(reso_corr.nonzero()[0]) == len(reso.nonzero()[0])
if not bad_time_txt:
    assert len_corr == (len(lv_time_stamps_corr.nonzero()[0]) + len(reso_corr.nonzero()[0]) + 1) 
else:  # first row of broken time.txt has non-zero first column value
    assert len(lv_time_stamps_corr.nonzero()[0]) - 1 == len(lv_tstamps_ms_shifted)


In [ ]:
if not(lv_time_stamps_corr[0] < lv_time_stamps_corr[1]):
    print("Warning: original first row belt (first column) entry of time.txt larger than first subsequent matched .txt time stamp!")
    print("Trying to dirty fix it...")
    # remove first row
    lv_time_stamps_corr = lv_time_stamps_corr[1:]
    reso_corr = reso_corr[1:]
    galvo_corr = galvo_corr[1:]
    lfp_corr = lfp_corr[1:]
    # mimic first row in other columns
    lfp_corr[0] = lv_time_stamps_corr[0]
    print("Success")

In [ ]:
#len(lv_time_stamps_corr)
len(galvo_corr)

In [ ]:
export_folder = "D:\\Downloads"

In [ ]:
lv_tstamp_corr_fname = os.path.splitext(lv_tstamp_fname)[0]+"_corr.txt"
print(lv_tstamp_corr_fname)

In [ ]:
export_fpath = os.path.join(export_folder, lv_tstamp_corr_fname)

In [ ]:
with open(export_fpath, "w") as f:
    for i_row in range(len(lv_time_stamps_corr)):
        f.write(f"{lv_time_stamps_corr[i_row]:.3f}\t{reso_corr[i_row]:.3f}\t{galvo_corr[i_row]:.3f}\t{lfp_corr[i_row]:.3f}\n")